In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
ema = pd.read_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/ema_mio202540.csv")

In [4]:
ema = ema.copy()

# -------------------------------
# 1. Calcular venta_costo_ema
# -------------------------------
ema["venta_costo_ema"] = (ema["EMA"] * ema["producto_costo_unitario"]).round(2)

# -------------------------------
# 2. Totales por sucursal
# -------------------------------
ema["venta_costo_tot"] = ema.groupby("store_name")["venta_costo_ema"].transform("sum")
ema["total_ema"] = ema.groupby("store_name")["EMA"].transform("sum")

# -------------------------------
# 3. Proporciones dentro de cada sucursal
# -------------------------------
ema["venta_costo%"] = (ema["venta_costo_ema"] / ema["venta_costo_tot"]).round(4)
ema["cantidad%"] = (ema["EMA"] / ema["total_ema"]).round(4)

# -------------------------------
# 4. Orden y acumulados
# -------------------------------
# Ordenar por costo dentro de cada sucursal
ema = ema.sort_values(["store_name", "venta_costo_ema"], ascending=[True, False])
ema["acumulado_costo"] = ema.groupby("store_name")["venta_costo%"].cumsum().round(4)

# Ordenar por cantidad dentro de cada sucursal
ema = ema.sort_values(["store_name", "EMA"], ascending=[True, False])
ema["acumulado_cantidad"] = ema.groupby("store_name")["cantidad%"].cumsum().round(2)

# Acumulado de desviación por sucursal
if "desviacion_ema%" in ema.columns:
    ema["acumulado_desviacion"] = (
        ema.groupby("store_name")["desviacion_ema%"].cumsum().round(2)
    )

# -------------------------------
# 5. Renombrar columnas (estilo reporte Odoo)
# -------------------------------
ema_def = ema.rename(
    columns={
        "año": "Año",
        "semana_num": "semana",
        "EMA": "Ema",
        "producto_costo_unitario": "Costo unitario",
        "venta_costo_ema": "Venta costo EMA",
        "venta_costo%": "Venta costo %",
        "acumulado_costo": "Acumulado costo",
        "desviacion_ema%": "Desviacion EMA %",
    }
)

# -------------------------------
# 6. Calcular desviación en dinero
# -------------------------------
if "Desviacion EMA %" in ema_def.columns:
    ema_def["Desviacion EMA"] = (
        ema_def["Venta costo EMA"] * ema_def["Desviacion EMA %"]
    ).round(1)



# Ejemplo de reglas (lo que viene de conf_acumulado_desviacion en Odoo) #PERILLA
reglas = [
    {"rango_ini": 0, "rango_fin": 0.8, "valor": 0},
    {"rango_ini": 0.8, "rango_fin": 1.5, "valor": 0.5},
    {"rango_ini": 1.5, "rango_fin": 3, "valor": 0.8},
    {"rango_ini": 3, "rango_fin": 2000, "valor": 1},
]

# aplicar reglas 0–0.8–1 como antes, pero sobre coef_ventas_norm


# Función que asigna el valor según reglas
def asignar_desviacion(coef, reglas):
    for r in reglas:
        if r["rango_ini"] <= coef <= r["rango_fin"]:
            return r["valor"]
    return np.nan  # si no entra en ningún rango

# Aplicar
ema_def["acumulado_desviacion_p"] = ema_def["coef_ventas"].apply(
    lambda x: asignar_desviacion(x, reglas)
)



# -------------------------------
# 7. Limpiar columnas innecesarias
# -------------------------------
if "Unnamed: 0" in ema_def.columns:
    ema_def = ema_def.drop(columns=["Unnamed: 0"])


# 8. Reglas acumulado costo combinado
# --- Configuración ---
def check_div(a, b):
    try:
        return a / b if b != 0 else 0
    except Exception:
        return 0

## PERILLA

conf_porcentajes_impacto = {
    "cantidad": 0.5,
    "costo": 0.3,
    "desviacion": 0.2,
    "cantidad_mayor": 0.7,
    "costo_mayor": 0.3,
}

# Valor de referencia en porcentaje (ejemplo: 10%) ##PERILLA
conf_acumulado_cantidad = 80
conf_acumulado_cantidad_desviacion = check_div(conf_acumulado_cantidad, 100)


# --- Cálculo en el DataFrame ---
def calcular_combinado(row):
    acumulado_cantidad_final = round(row["acumulado_cantidad"], 2)
    acumulado_costo_final = round(row["Acumulado costo"], 2)
    acumulado_desviacion_p_final = round(row["acumulado_desviacion_p"], 2)

    if acumulado_cantidad_final >= conf_acumulado_cantidad_desviacion:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad_mayor"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo_mayor"]
        )
    else:
        return (
            acumulado_cantidad_final * conf_porcentajes_impacto["cantidad"]
            + acumulado_costo_final * conf_porcentajes_impacto["costo"]
            + acumulado_desviacion_p_final * conf_porcentajes_impacto["desviacion"]
        )

# Nueva columna en tu df
ema_def["acumulado_combinado"] = ema_def.apply(calcular_combinado, axis=1).round(2)

ema_def["acumulado_combinado"] = (
    ema_def.groupby("store_name")["acumulado_combinado"]
    .transform(lambda x: x / x.max())
).round(2)


def clasificar_pareto(valor):
    if valor <= 0.50:
        return "AAA"
    elif valor < 0.8:
        return "A"
    elif valor < 0.95:
        return "B"
    else:
        return "C"

ema_def["Clasificacion"] = ema_def["acumulado_combinado"].apply(clasificar_pareto)


In [5]:
ema_ord = ema_def[["store_name","product_ref","Año","semana","Ema",
"Costo unitario","Venta costo EMA","Venta costo %","Acumulado costo","Desviacion EMA",
"Desviacion EMA %","acumulado_cantidad","coef_ventas","acumulado_desviacion","acumulado_desviacion_p","acumulado_combinado","Clasificacion"]]

ema_ord = ema_ord[~(ema_ord["store_name"]=="0")]

In [6]:
ema_ord["acumulado_combinado"].describe()

count    432896.000000
mean          0.978644
std           0.087425
min           0.020000
25%           1.000000
50%           1.000000
75%           1.000000
max           1.000000
Name: acumulado_combinado, dtype: float64

In [7]:
ema_ord[(ema_ord["store_name"]=="SUCURSAL NORTE") & (ema_ord["product_ref"]=="DAE02286025")]

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion
284262,SUCURSAL NORTE,DAE02286025,2025.0,38,1.631310,97279.53,158693.07,0.0007,0.2236,1.400420e+10,88247.06,0.50,0.0,8867202.41,0.0,0.36,AAA
284263,SUCURSAL NORTE,DAE02286025,2025.0,39,1.305048,97279.53,126954.46,0.0006,0.2647,1.167981e+05,0.92,0.56,13.0,9602920.97,1.0,0.62,A
284264,SUCURSAL NORTE,DAE02286025,2025.0,40,1.044038,97279.53,101563.53,0.0005,0.3183,1.167981e+05,1.15,0.63,14.0,11524154.30,1.0,0.68,A
284265,SUCURSAL NORTE,DAE02286025,2025.0,41,0.835230,97279.53,81250.78,0.0004,0.3765,1.040010e+05,1.28,0.67,15.0,13586829.66,1.0,0.72,A
284250,SUCURSAL NORTE,DAE02286025,2025.0,26,0.320000,97279.53,31129.45,0.0001,0.6462,0.000000e+00,0.00,0.90,0.0,28841389.08,0.0,0.91,B
284251,SUCURSAL NORTE,DAE02286025,2025.0,27,0.256000,97279.53,24903.56,0.0001,0.6965,0.000000e+00,0.00,0.90,0.0,33078156.23,0.0,0.93,B
284252,SUCURSAL NORTE,DAE02286025,2025.0,28,0.204800,97279.53,19922.85,0.0001,0.7503,9.016052e+08,45254.83,0.90,0.0,39597981.61,0.0,0.96,C
284253,SUCURSAL NORTE,DAE02286025,2025.0,29,0.163840,97279.53,15938.28,0.0001,0.8077,9.199320e+08,57718.40,0.90,0.0,43515706.35,0.0,0.97,C
284254,SUCURSAL NORTE,DAE02286025,2025.0,30,0.147456,97279.53,14344.45,0.0001,0.8355,9.671040e+08,67420.08,0.90,0.0,45346147.69,0.0,0.98,C
284255,SUCURSAL NORTE,DAE02286025,2025.0,31,0.132710,97279.53,12909.97,0.0001,0.8638,9.112994e+08,70588.81,0.90,0.0,47609358.86,0.0,0.99,C


## NIVEL DE SERVICIO

In [8]:
# Definimos las condiciones para la meta, basadas en la clasificación
condiciones_meta = [
    (ema_ord['Clasificacion'] == 'AAA'),
    (ema_ord['Clasificacion'] == 'A'),
    (ema_ord['Clasificacion'] == 'B'),
    (ema_ord['Clasificacion'] == 'C')
]

# Definimos los valores de la meta que se asignarán a cada clasificación
metas = [0.999, 0.98, 0.95, 0.50]

ema_ord = ema_ord.copy()

# Asignamos la nueva columna 'Meta' usando np.select
ema_ord.loc[:, 'Nivel de servicio'] = np.select(condiciones_meta, metas, default=np.nan)

In [9]:
ema_ord.to_csv("/home/donsson/proyectos/MODELO ABASTECIMIENTO/csvsalidas/clasificacion_mia2025.csv")

bq = ema_ord[ema_ord["store_name"]=="SUCURSAL BARRANQUILLA"].sort_values(by=["Ema"],ascending=False)

bq.head((5))

,store_name,product_ref,Año,semana,Ema,Costo unitario,Venta costo EMA,Venta costo %,Acumulado costo,Desviacion EMA,Desviacion EMA %,acumulado_cantidad,coef_ventas,acumulado_desviacion,acumulado_desviacion_p,acumulado_combinado,Clasificacion,Nivel de servicio
205766,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,32,63.636743,13645.86,868378.09,0.0009,0.0987,138940.5,0.16,0.00,2.117647,0.16,0.8,0.22,AAA,0.999
205767,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,33,57.109394,13645.86,779306.80,0.0008,0.1057,148068.3,0.19,0.00,3.416000,0.35,1.0,0.26,AAA,0.999
205775,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,41,55.274361,13645.86,754266.19,0.0008,0.1097,143310.6,0.19,0.01,4.146526,0.54,1.0,0.28,AAA,0.999
205768,SUCURSAL BARRANQUILLA,DAB02570025,2025.0,34,54.487515,13645.86,743529.00,0.0008,0.1121,163576.4,0.22,0.01,3.605911,0.76,1.0,0.28,AAA,0.999
247019,SUCURSAL BARRANQUILLA,DAB14570025,2025.0,41,49.942082,9917.17,495284.12,0.0005,0.1996,188208.0,0.38,0.01,4.964789,1.14,1.0,0.30,AAA,0.999


In [10]:
df_mio = ema_ord[["store_name","product_ref","semana","Ema","acumulado_combinado","Clasificacion"]]


In [11]:
conteo = pd.crosstab(df_mio["store_name"], df_mio["Clasificacion"])
conteo

Clasificacion,A,AAA,B,C
store_name,,,,
PRINCIPAL COTA,1467,396,1782,50467
SUCURSAL BARRANQUILLA,1873,619,2226,49394
SUCURSAL BUCARAMANGA,2027,641,2226,49218
SUCURSAL CALI,1779,489,2149,49695
SUCURSAL CALLE 6,1731,745,2242,49394
SUCURSAL MEDELLIN,1521,512,2026,50053
SUCURSAL NORTE,1630,469,2041,49972
SUCURSAL VALLADOLID,1458,413,1891,50350
